<a href="https://colab.research.google.com/github/Torresbr197518/gestao_rh/blob/master/Eventos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import numpy as np
from datetime import datetime


AttributeError: ignored

In [90]:
lista= evento.events[evento.events['status']!='Full']['leagueid'].values
#type(evento.list_leagues)

In [94]:
evento.list_leagues=set(lista)

In [2]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [194]:
class Events:
  def __init__(self):
    self.url = 'https://www.totalcorner.com/match/today'
    self.load()
    self.scrap()
    self.get_leagues()

  def scrap (self):
     res = requests.get(self.url)  
     self.soup = BeautifulSoup(res.content,'lxml')

  def get_leagues (self) :
      self.list_leagues = []
      for c in self.soup.find_all('a'):
          #print (c['href'].split("/").str.contains('view', case = False))
          ref = 'league/view/' in c['href']
          if ref==True:
              self.list_leagues.append(int(c['href'].strip("league/view/")))  

      self.list_leagues = set(self.list_leagues)

  def get_events(self, rows, leagueid, page):
        league = 'https://www.totalcorner.com/league/view/'
        url = '{}{}/page:{}'.format(league,leagueid, page)
        res = requests.get(url)
        soup = BeautifulSoup(res.content,'lxml')
        tables = soup.find_all('tr')
        print (url)
        for table in tables[1:]:
          line = table.find_all('td')
          x, err = self.preparar(8,2021 , leagueid, line, rows)
          if err==True:
            break
        #self.save()
        time.sleep(0.51)    

  def transformDate(self, ano, date):
    err = False
    try:  
      dia = int(date[3:5].strip())
      mes = int(date[:2].strip())
      if mes ==12 :
        ano = ano - 1
      hora = int(date[6:8].strip())
      min = int(date[9:11].strip())
      date = datetime(ano,mes,dia,hora,min)
    except:
      print ('Erro de conversão :'+date)
      err = True
    return date, err

  def preparar (self, mes_inicio,ano,leagueid, row, rows):
    err=False
    if row[0].text.find('/')!=-1: 
      date, err = self.transformDate(ano,row[0].text)
      status = row[1].text.strip()
      eventid = row[14].find_all('a')[1]['href'].replace('/match/odds-handicap/', '')
      if len(self.events[(self.events['eventid']==eventid) & (self.events['status']!='Full')])==0:
          home = row[2].text.strip()
          away = row[4].text.strip()
          score = row[3].text.strip().split(' - ')
          homescore = score[0].strip()
          awayscore = score[1].strip()
          corner = row[6].text.strip().split(' - ')
          homecorner = corner[0].strip()
          awaycorner = corner[1].strip()
          halfcorner = row[7].text.strip().split(' - ')
          homehalfcorner = halfcorner[0].strip()
          awayhalfcorner = halfcorner[1].strip()
          dados = [date, eventid, status,home, away, homescore, awayscore, homecorner, awaycorner,homehalfcorner,awayhalfcorner, leagueid]
          rows = rows.append(dados)
          print(dados)
    return rows, err  

  def get_list(self):
    for leagueid in self.list_leagues:
      rows = []
      for page in range (1,5):
        ano = 2021
        mes_inicio = 10
        self.get_events(rows, leagueid, page)
      
      columns = ['date', 'eventid', 'status','home', 'away','homescore', 'awayscore', 'homecorner', 'awaycorner','homehalfcorner','awayhalfcorner', 'leagueid']

      self.events = self.events.append(pd.DataFrame(rows,columns = columns), ignore_index=False)

      self.events = self.events.drop_duplicates(subset=['eventid'], keep='last')
    #print(rows)
    self.save()

  def load (self):
    self.events =pd.read_excel('eventos.xlsx')

  def save (self):
    self.events.to_excel('eventos.xlsx')

In [195]:
evento = Events()

In [198]:
evento.events = evento.events.drop_duplicates(subset=['eventid'], keep='last')

In [ ]:
evento.get_list()

In [112]:
evento.get_leagues()

In [ ]:
evento.get_list()

In [199]:
evento.events

,Unnamed: 0,date,eventid,status,home,away,homescore,awayscore,homecorner,awaycorner,homehalfcorner,awayhalfcorner,leagueid
0,0.0,2021-10-10 02:00:00,108606227,NaN,Atletico Huila,Atletico Nacional,0,0,0,0,0,0,145
1,1.0,2021-10-10 06:00:00,109015672,NaN,Khukh Chononuud Women,Khad Women,0,0,0,0,0,0,19473
2,2.0,2021-10-10 04:00:00,109015670,NaN,Shine Yarmag Women,Tuuliin Tom Tulnuud Women,0,0,0,0,0,0,19473
3,3.0,2021-10-10 02:00:00,109015662,NaN,UBC FC Women,Mongoliin Temuulel Women,0,0,0,0,0,0,19473
4,4.0,2021-10-10 05:00:00,108991238,NaN,FC Osaka,Sony Sendai,0,0,0,0,0,0,422
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,NaN,2021-11-05 15:00:00,95135907,Full,Colchagua CD,Lautaro de Buin,0,2,4,4,3,3,255
156,NaN,2021-11-02 15:00:00,95026750,Full,Deportes Vallenar,AC Colina,3,2,2,6,1,3,255
157,NaN,2021-11-01 15:00:00,95000377,Full,San Antonio Unido,Iberia,1,6,7,8,6,2,255
158,NaN,2021-11-01 15:00:00,95000375,Full,Lautaro de Buin,Deportes Concepcion,2,1,3,5,2,2,255


In [205]:
evento.events[(evento.events.status=='Full') & (evento.events['home']=='FC Osaka')]

,date,eventid,status,home,away,homescore,awayscore,homecorner,awaycorner,homehalfcorner,awayhalfcorner,leagueid
27,2021-09-05 05:56:00,372964944,Full,FC Osaka,Honda FC,1,2,5,8,3,2,422
4399,2021-07-25 06:00:00,105613406,Full,FC Osaka,Tokyo Musashino City FC,0,2,7,7,3,2,422
4409,2021-07-11 06:00:00,104991284,Full,FC Osaka,Veertien Mie,1,1,7,6,2,4,422
4417,2021-06-27 05:00:00,104395827,Full,FC Osaka,Nara Club,1,1,6,2,4,0,422
4460,2021-04-24 05:00:00,101703879,Full,FC Osaka,Iwaki SC,1,1,8,9,2,4,422
4472,2021-03-27 04:00:00,100500609,Full,FC Osaka,Reinmeer Aomori,1,0,10,1,5,1,422
4479,2021-03-14 04:00:00,99951047,Full,FC Osaka,Kariya,3,0,12,1,8,0,422
4486,2021-11-22 04:00:00,95740312,Full,FC Osaka,Nara Club,2,1,8,3,4,0,422
4494,2021-11-14 04:00:00,95473756,Full,FC Osaka,Kochi United,0,2,8,3,1,1,422
4511,2021-10-03 05:00:00,93881015,Full,FC Osaka,Maruyasu Okazaki,0,0,5,4,0,2,422


In [204]:
evento.events = evento.events[evento.events.columns.drop(['Unnamed: 0'])]

#evento.events.columns.drop (['Unnamed: 0','Unnamed: 0.1'])      

In [200]:
Mandante = evento.events[evento.events['home']=='AC Milan (zohan) Esports']
Visitante = evento.events[evento.events['away']=='AC Milan (zohan) Esports']

Todos = pd.DataFrame(Mandante)

Todos= Todos.append(Visitante)

#Todos.set_index('date', inplace=True)


#df = pd.concat(pd.Series(Mandante), pd.Series(Visitante), Axis=1)
print(Todos)
#Todos = pd.concat(Mandante,Visitante)



Empty DataFrame
Columns: [Unnamed: 0, date, eventid, status, home, away, homescore, awayscore, homecorner, awaycorner, homehalfcorner, awayhalfcorner, leagueid]
Index: []


In [189]:

#evento.events.drop(index=0)
evento.save()
##evento.events.drop(events.columns[[1]], axis=1)

In [9]:
import sys

rows = []
events =pd.DataFrame(rows,columns = columns)

list_leagues = Lista_eventos ()

    
    clear_output()
    print(events.tail(1))

                    date    eventid  ... awayhalfcorner leagueid
1478 2021-09-20 19:00:00  108209611  ...              1    17914

[1 rows x 12 columns]


In [ ]:
events[(events['status']!='Full') & (events['date']<='2021-10-10')]

In [7]:
list_leagues = Lista_eventos ()

for leagueid in list_leagues:
    rows = []

    for page in range (1,3):
      ano = 2021
      mes_inicio = 5
      Get_events(rows, leagueid, page)

    events =events.append(pd.DataFrame(rows,columns = columns), ignore_index=True)

    events = events.drop_duplicates(subset=['eventid'], keep='last')

    #events.to_excel('eventos.xlsx')
    from IPython.display import clear_output
    
    clear_output()
    print(events.tail(1))

NameError: ignored

In [ ]:
len(events)

1740

In [ ]:
events[events['eventid']=='102144999'].count()[1]

1

In [ ]:
events.drop(events.columns[0], axis=1, inplace=True)

In [ ]:
list_leagues=events[events['status']!='Full']['leagueid']

In [ ]:
events[events['date']>'01/01/2021']

TypeError: ignored

In [ ]:
events['eventid'] = events['eventid'] .str.lstrip()

In [ ]:
eventsactive
#print()
#events

,date,eventid,status,home,away,homescore,awayscore,homecorner,awaycorner,homehalfcorner,awayhalfcorner,leagueid


In [ ]:
events[events['leagueid']==13321]

,date,eventid,status,home,away,homescore,awayscore,homecorner,awaycorner,homehalfcorner,awayhalfcorner,leagueid


In [ ]:
events.drop(events.columns[0], axis=1, inplace=True)

In [ ]:
events.set_index(['date'], drop=False, inplace=True)

In [ ]:
https://www.totalcorner.com/league/view/150
https://www.totalcorner.com/league/view/852

In [ ]:
odds = getodds(eventid)
odds = odds.replace(np.nan, 0)
results = []
for item in odds['Score'].unique():
    m = int(odds[odds['Score'] == item].iloc[-1]['Minute'])
    w = round(odds[odds['Score'] == item].iloc[-1]['Win'],2)
    d = round(odds[odds['Score'] == item].iloc[-1]['Draw'],2)
    l = round(odds[odds['Score'] == item].iloc[-1]['Lose'],2)
    results = results.append([m, w, d, l])


In [ ]:
print(events[events['homescore']>events['awayscore']].count())
print(events[events['homescore']==events['awayscore']].count())
print(events[events['homescore']<events['awayscore']].count())

date              2285
eventid           2285
status            2285
home              2285
away              2285
homescore         2285
awayscore         2285
homecorner        2285
awaycorner        2285
homehalfcorner    2285
awayhalfcorner    2285
leagueid          2285
dtype: int64
date              1166
eventid           1166
status            1166
home              1166
away              1166
homescore         1166
awayscore         1166
homecorner        1166
awaycorner        1166
homehalfcorner    1166
awayhalfcorner    1166
leagueid          1166
dtype: int64
date              1152
eventid           1152
status            1152
home              1152
away              1152
homescore         1152
awayscore         1152
homecorner        1152
awaycorner        1152
homehalfcorner    1152
awayhalfcorner    1152
leagueid          1152
dtype: int64


In [ ]:
events =events['date'].dropna(how='all')

In [ ]:
events = pd.read_excel('eventos.xlsx')

In [ ]:
events

,date,eventid,status,home,away,homescore,awayscore,homecorner,awaycorner,homehalfcorner,awayhalfcorner,leagueid,HomePoints,AwayPoints
0,2021-08-31 01:30:00,106736074,,Fortaleza FC,Cuiaba,0,0,0,0,0,0,129,1,1
1,2021-08-30 23:00:00,106736090,,Fluminense,EC Bahia,0,0,0,0,0,0,129,1,1
2,2021-08-30 00:30:00,106736093,11 ',Bragantino,Atletico Mineiro,0,0,0,2,0,2,129,1,1
3,2021-08-29 22:15:00,106736100,Full,Atletico Goianiense,Internacional,0,0,6,6,5,1,129,1,1
4,2021-08-29 20:00:00,106736083,Full,EC Juventude,Sao Paulo,1,1,3,8,1,3,129,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1735,2021-05-16 23:00:00,102695381,Full,Orlando Pride Women,Washington Spirit Women,1,1,7,5,3,2,1264,1,1
1736,2021-05-16 03:00:00,102631034,Full,Reign FC Women,North Carolina Courage Women,0,0,2,9,2,3,1264,1,1
1737,2021-05-16 00:00:00,102651168,Full,NJ/NY Gotham FC Women,Houston Dash Women,1,0,2,6,1,5,1264,3,0
1738,2021-05-15 22:00:00,102644767,Full,Racing Louisville FC Women,FC Kansas City Women,0,0,7,1,2,1,1264,1,1


In [ ]:
events[events['away']=='Napoli (Yoshi) Esports']

,date,eventid,status,home,away,homescore,awayscore,homecorner,awaycorner,homehalfcorner,awayhalfcorner,leagueid


In [ ]:
events['HomePoints'] = np.where(events['homescore']>events['awayscore'],3, np.where(events['homescore']==events['awayscore'],1,0))
events['AwayPoints'] = np.where(events['homescore']<events['awayscore'],3, np.where(events['homescore']==events['awayscore'],1,0))

In [ ]:
for index, row in events.iterrows():
    HSUM = events[(events['home']==row["home"])]['HomePoints'].sum()+events[(events['away']==row["home"])]['AwayPoints'].sum()
    ASUM = events[(events['home']==row["away"])]['HomePoints'].sum()+events[(events['away']==row["away"])]['AwayPoints'].sum()
    print(HSUM, ASUM, row["home"], row["away"], row["homescore"],row["awayscore"])
    print()
    #if index >100: break

In [ ]:
#select = events[(events['home'].str.contains('Flamengo', case = False) & events['away'].str.contains('Fluminense', case = False) )]

select = events[events['leagueid']==85]
select.groupby(['leagueid','homescore','awayscore']).sum()



HomePoints  AwayPoints
leagueid homescore awayscore                        
85       0         0                 144         144
                   1                   0         378
                   2                   0         261
                   3                   0          87
                   4                   0          30
                   5                   0          12
                   6                   0           3
                   7                   0           3
         1         0                 666           0
                   1                 233         233
                   2                   0         384
                   3                   0         186
                   4                   0          48
                   5                   0          15
                   6                   0           6
         2         0                 546           0
                   1                 807           0
                   2                 146         146
                   3                   0         144
                   4                   0          30
                   5                   0          12
                   6                   0           3
         3         0                 366           0
                   1                 363           0
                   2                 267           0
                   3                  33          33
                   4                   0          24
                   5                   0           6
                   6                   0           3
         4         0                 162           0
                   1                 180           0
                   2                  90           0
                   3                  51           0
                   4                   6           6
         5         0                  45           0
                   1                  54           0
                   2                  24           0
                   3                   9           0
         6         0                   9           0
                   1                  12           0
                   2                   9           0
                   3                   3           0
                   4                   3           0
         7         0                   3           0
                   1                   9           0
                   2                   3           0

In [16]:
def getodds (eventid):

    #odds_full = soup.find_all('table', {'id':['odds_full']})
    #odds_half = soup.find_all('table', {'id':['odds_half']})
    #goals_full = soup.find_all('table', {'id':['goals_full']})
    #goals_half = soup.find_all('table', {'id':['goals_half']})
    #corners_full = soup.find_all('table', {'id':['corners_full']})
    #corners_half = soup.find_all('table', {'id':['corners_half']})
    df = pd.DataFrame()

    res = requests.get('https://www.totalcorner.com/match/odds-handicap/{}'.format(eventid))
    soup = BeautifulSoup(res.content,'lxml')
    table = soup.find_all('table', {'id':['odds_full']})
    if len(table)>0:
      print(eventid)
      df = pd.DataFrame(pd.read_html(str(table))[0])
      df.columns= ['Minute', 'Score', 'Win', 'Draw', 'Lose','Time']
      df['Minute']= df['Minute'].str.replace("'","")
      #df = df.dropna()
      #df.iloc[-1]
    return df

In [17]:
#events['oddsfull']
#events['oddsfull']= getodds(events['eventid'])
getodds(1866046121)


1866046121


,Minute,Score,Win,Draw,Lose,Time
0,05,4 - 0,1.007,29.00,29.000,04-17 03:30
1,05,4 - 0,1.008,26.00,26.000,04-17 03:30
2,half,4 - 0,1.010,26.00,26.000,04-17 03:29
3,04,4 - 0,1.013,23.00,23.000,04-17 03:29
4,04,4 - 0,1.015,21.00,21.000,04-17 03:28
5,03,3 - 0,1.045,11.00,15.000,04-17 03:28
6,03,3 - 0,1.050,11.00,15.000,04-17 03:27
7,03,2 - 0,1.167,7.50,9.000,04-17 03:27
8,02,2 - 0,1.167,7.50,8.500,04-17 03:27
9,02,2 - 0,1.182,7.50,8.500,04-17 03:26


In [ ]:
url ='100441873'
res = requests.get('https://www.totalcorner.com/match/odds-handicap/{}'.format(url))
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')

odds_full = soup.find_all('table', {'id':['odds_full']})
odds_half = soup.find_all('table', {'id':['odds_half']})
goals_full = soup.find_all('table', {'id':['goals_full']})
goals_half = soup.find_all('table', {'id':['goals_half']})
corners_full = soup.find_all('table', {'id':['corners_full']})
corners_half = soup.find_all('table', {'id':['corners_half']})


df = pd.DataFrame()
df = df.append(pd.read_html(str(odds_full)))
df = df['']
df.iloc[-1]

matchstatus = td.find_all('td', {'class':['text-center match_status']})
  matchstatus = matchstatus[0]
  matchstatus = matchstatus.text.strip()

  home = td.find_all('td', {'class':['text-right match_home','text-right']})
  homeid = home[0].find_all('a', href=True)
  homeid = homeid[0]['href']
  home = home[0]
  home = home.text.strip()
  homeid = homeid

  score = td.find_all('td', {'class':['text-center match_goal']})
  score = score[0].text.split(' - ')
  
  away = td.find_all('td', {'class':['text-left match_away','text-right']})
  awayid = away[0].find_all('a', href=True)
  awayid = awayid[0]['href']
  away = away[0]
  away = away.text.strip()
  awayid = awayid

  matchcorner = td.find_all('span', {'class':['span_match_corner']})
  if len(matchcorner)>0:
    matchcorner = matchcorner[0].text.split(' - ')
  
  halfcorner = td.find_all('span', {'class':['span_half_corner']})
  if len(halfcorner)>0:
    halfcorner = halfcorner[0].text
    halfcorner = halfcorner[halfcorner.find("(")+1:halfcorner.find(")")].split('-')
  
  eventid = ''
  #td.find_all('td', {'class':['text-center td_analysis','text-center']})
  eventstats = ''
  eventodss =''

  #if len(eventid)>0:
  #  eventid = eventid[0].find_all('a', href=True)
  #  eventstats = eventid[0]['href']
  #  eventodss = eventid[1]['href']
  
  #columns = ['league','matchstatus','home','homeid','scorehome','scoreaway','away','awayid','matchcornerhome','matchcorneraway','halfcornerhome','halfcorneraway','eventstats','eventodss']
  data = [[league, matchstatus, home, homeid,score[0], score[1], away, awayid, matchcorner[0], matchcorner[1], halfcorner[0], halfcorner[1], eventstats, eventodss]] 
  print(data)
  ## Create the pandas DataFrame 
  #df = pd.DataFrame(data, columns = columns)   

In [ ]:
!pip3 install MetaTrader5-5.0.34-cp39-cp39-win_amd64.whl

ERROR: MetaTrader5-5.0.34-cp39-cp39-win_amd64.whl is not a supported wheel on this platform.


In [ ]:
pip install py -m venv <DIR>

SyntaxError: ignored

In [ ]:
pip install --upgrade MetaTrader5

ERROR: Could not find a version that satisfies the requirement MetaTrader5 (from versions: none)
ERROR: No matching distribution found for MetaTrader5


In [ ]:
cd Mydrive


[Errno 2] No such file or directory: 'Mydrive'
/root


In [ ]:
 !pwd 

/root


In [ ]:
cd colab Notebooks


[Errno 2] No such file or directory: 'colab Notebooks'
/root


In [ ]:
cd 'drive/My Drive'

[Errno 2] No such file or directory: 'drive/My Drive'
/root


In [ ]:
md Rafael

SyntaxError: ignored

In [ ]:
%cd sample_data

[Errno 2] No such file or directory: 'sample_data'
/root
